# Theoretical Concepts

### Framework vs library

A `library` is essentially a set of functions that you can call, these days usually organized into classes. Each call does some work and returns control to the client.

A `framework` embodies some abstract design, with more behavior built in. In order to use it you need to insert your behavior into various places in the framework either by subclassing or by plugging in your own classes. The framework's code then calls your code at these points.

Other sources: https://ed.team/blog/framework-vs-libreria

### Templates vs Static Files

`Template files` are HTML pages that are presented to your users during the authentication process. The pages prompt users for authentication information, such as user names and passwords, or present information to users, such as one-time passwords, status, or errors.

You can customize any of the HTML pages by exporting, modifying, and importing its corresponding template file. Each template file uses one or more specific macros. Source :https://www.ibm.com/docs/en/sva/9.0.5?topic=settings-template-files

`Static files` are typically files such as scripts, CSS files, images, etc... that aren't server-generated, but must be sent to the browser when requested.

# Standard Web Technologies

### User? User Agent? and Client? Contrast
* `User` is the person who actually make use of the computer and computer related services
* `User agent` can be described as a browser or an mobile application. *Chrome*, *Mozilla Firefox*
* `clients` can be visualized as the application code. It is the code running at the end user side in order to 

Theory about user and agent:
https://medium.com/@winma.15/user-user-agent-and-client-what-differs-c2a31b3066f

# FastAPI

* To make the `requirement.txt` file that contains the packages we've installed
  * Type in terminal 
    ```cmd
    pip freeze > requirement.txt
    ```


## First Steps

The simplest FastAPI file could like this

* In the file `main.py`
    ```python
    from fastapi import FastAPI

    app = FastAPI()

    @app.get("/")
    async def root():
        return {"Message": "Hello World"}
    ```
* run `uvicorn main:app --reload`

`Uvicorn` creates the interface to an application connect to server thought a gateway. Formally `Uvicorn` is an ASGI (*Asynchronous Server Gateway Interface*) web server implementation for Python.

By default *port* is `8000` and the  `http` *protocol*, `127.0.0.1` point out the local server.


> The ASGI interface has two sides: the `server` or `gateway` side, and the `application` or `framework` side. The server side invokes a callable object that is provided by the application side

>  A gateway is a hardware device that acts as a "gate" between two networks. It may be a router, firewall, server, or another device that enables traffic to flow in and out of the network.
>
> While a gateway protects the nodes within network, it is also a node itself. The gateway node is considered to be on the `edge` of the network as all data must flow through it before coming in or going out of the network. It may also translate data received from outside networks into a format or protocol recognized by devices within the internal network.
> 
> A *router* is a common type of gateway used in home networks. It allows computers within the local network to send and receive data over the Internet. 
> 
> A *firewall* is a more advanced type of gateway, which filters inbound and outbound traffic, disallowing incoming data from suspicious or unauthorized sources. A proxy server is another type of gateway that uses a combination of hardware and software to filter traffic between two networks. For example, a proxy server may only allow local computers to access a list of authorized


More of uvicorn in https://www.uvicorn.org/

In the code `main` is the name of the file that contain the application `app` create inside of `main.py`.

This `app` is an instance of the `FastAPI()` class. This will be the main point of interaction to create all our API.

In the `app` we can define the *path*, *endpoint* or a *route* and the operation `get`, `delete`, `post`, `put` and others

* path is the last part of the URL starting from the first `/`
  * e.g. for https://www.lambdatest.com/free-online-tools/url-parse, `/free-online-tools/url-parse` is the path

* With the decoration `@` we can pass the ``app``, the operation `get` and the route `/`. For instance, `@app.get('/')`

* Below this decoration we define the `function` (can be sync or async) and what we want to `return`.


`--reload` make the server restart after code changes. Only use for development.

souces:https://peps.python.org/pep-3333/ ,https://asgi.readthedocs.io/en/latest/, https://www.uvicorn.org/server-behavior/


If we wan to see the documentation of our API, go to http://127.0.0.1:8000/docs or  http://127.0.0.1:8000/redocs

#### OpenAPI

FastAPI generates a "schema" with all your API using the OpenAPI standard for defining APIs.

To see it http://127.0.0.1:8000/openapi.json

### Request

If we want to know the URL, Headers, query parameters, Path Parameters, and so on of a *route*,  `Request` is the solution

```python
app = FastAPI()
@app.get("/books")
async def get_request_object(request: Request):
    # This request only get some information of the server
    # get the URL
    print(request.url)
    # the url for something
    print(request.url_for('static', path = ''))
    # get the host of the client
    print(request.client.host)
    return {"path": request.url.path, 'host':request.client.host}
```

### Templates

##### Jinja

`Jinja` is a templating engine

*Special placeholders* in the template allow writing code similar to Python syntax. Then the template is passed data to render the final document.

Example:
* In the file `app.py`
    ```python
    from fastapi import FastAPI, Request
    from fastapi.responses import HTMLResponse
    from fastapi.staticfiles import StaticFiles
    from fastapi.templating import Jinja2Templates

    app = FastAPI()

    app.mount("/static", StaticFiles(directory="static"), name="static")

    templates = Jinja2Templates(directory="templates")

    @app.get("/items/{id}", response_class=HTMLResponse)
    async def read_item(request: Request, id: str):
        url_path_id = request.url_for('static', path='/img/{}.webp'.format(id))
        context = {"request": request, "id": id, "url_path_id": url_path_id}
        return templates.TemplateResponse("item.html", context=context)
    ```
* In the file `HTML`

    ```html
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Item {{id}}</title>
    </head>
    <body>
        <img src= {{url_path_id}}  alt="">
    </body>
    </html>

    ```
source :https://jinja.palletsprojects.com/en/3.1.x/templates/

The tree is

* static
  * css
    * style.css
  * img
    * 41ee337dddc249baa79bae7400a4c1f2_9366.webp
    * ...
  * script
    * script.js
* templates
  * item.html
* app.py

When we go to http://127.0.0.1:8000/items/b888f601abe04096baadac08011583ad_9366

This will render a page with pics

### Jinja Templates

*Delimiters*
* `{% ... %}`: for Statements
* `{{ ... }}`: for Expressions to print to the template output
* `{# ... #}`: for Comments not included in the template output
For the *expressions* we can access to attribute of the dict with `.` or `['']`, e.g. `{{ item.name }}`. If the attribute doesn't exist this return null or none, an empty value.

*Filters*



The methods are applied from the left to right separed by `|`, e.g. `{{ name | upper }}`

More filters in https://jinja.palletsprojects.com/en/3.1.x/templates/#builtin-filters

*Test*

Run some part of code if x meet some conditions

```jinja
{% if x>50 %}
<p>Greater than 50</p>
{% else %}
<p>Less or equal than 50</p>
{% endif %}
```
More test in https://jinja.palletsprojects.com/en/3.1.x/templates/#builtin-tests

#### Template Inheritance

The template inheritance allow us to build a base template that are common in our site and defined like a block children templates that can override it.

*Base templates*



Defines a simple HTML skeleton document that you might use in our site

```jinja

<!DOCTYPE html>
<html lang="en">
<head>
    {% block head %}
    <link rel="stylesheet" href="style.css" />
    <title>{% block title %}{% endblock %} - My Webpage</title>
    {% endblock %}
</head>
<body>
    <div id="content">{% block content %}{% endblock %}</div>
    <div id="footer">
        {% block footer %}
        &copy; Copyright 2008 by <a href="http://domain.invalid/">you</a>.
        {% endblock %}
    </div>
</body>
</html>
```

1. `{% block %}` define the the block that the child template can fill, can be override as well. Each block is named, e.g. `head`, `title`, `content`, `footer` and at the end we need to close it with `{% endblock %}`
2. As we can see we can have a block inside a block, this called *nested blocks*

*Child Templates*

```jinja
{% extends "base.html" %}
{% block title %}Index{% endblock %}
{% block head %}
    {{ super() }}
    <style type="text/css">
        .important { color: #336699; }
    </style>
{% endblock %}
{% block content %}
    <h1>Index</h1>
    <p class="important">
      Welcome to my awesome homepage.
    </p>
{% endblock %}
```

1. To use a base template we need to pull it with `{% extends "base.html" %}`
2. Then we can call the block that we fill or override. In this case the blocks are `head`, `title` and `content`. `footer` was not called, so this block is filled with the default content (the content in the base template).
3. In `title` block the content is filled (with Index), while for `head` and `content` blocks the contents are overriden
4. It’s possible to render the contents of the parent block by calling `super()`

With `Jinga` we can include a file HTML in a HTML

```html
{% include 'out.html' %}
```

#### HTMX

Htmx is a library that allows you to access modern browser features directly from HTML, rather than using javascript.


* In the `index.html` file
  ```html
  <button
    hx-get="/items"
    hx-trigger="click"
    hx-target="#parent-div"
    hx-swap="beforeend"
    name="button"
    id="1321"
  >
    Click Me!
  </button>
  <div id="parent-div">
    <h1>The new Here</h1>
  </div>
  <script src="https://unpkg.com/htmx.org@1.8.0"></script>
  ```
* In the `app.py`
  ```python

  from typing import Optional
  from fastapi import FastAPI, Request, Header
  from fastapi.responses import HTMLResponse
  from fastapi.staticfiles import StaticFiles
  from fastapi.templating import Jinja2Templates

  app = FastAPI()

  app.mount("/static", StaticFiles(directory="static"), name="static")

  templates = Jinja2Templates(directory="templates")

  @app.get('/items', response_class=HTMLResponse)

  async def main(request:Request, hx_trigger :Optional[str]=Header(None)):
    # hx_trigger contain the id value
    if hx_trigger == '1321':
        return "<h1>Title 1321</h1>"
    context = {'request':request}
    return templates.TemplateResponse('index.html', context=context)
  ```



The `<script src="https://unpkg.com/htmx.org@1.8.0"></script>` is need to include in html file

The button issues a request to `/items` (in the operation *get*) when this is *trigger* (*active*). And we choice what the parameters that we sent to header of the `/items`. In this case we sent the values of the id of the element that trigger, that is, the buttom. `hx_trigger` contain this value. (more information https://htmx.org/docs/#requests).

In `hx-target` we set the element where the response (return) of the request sent by the button

In `hx-swap` we set if the behaviour of the response, sometimes we want the response replace the target element (hx-target), so in this case we set `outerHTML`. Or include inside the target element `innerHTML` or include + order (`afterend`)


In the file `py` we set the `hx_trigger` as optinal paremeter since, this is passed when the buttom trigger or activate.

We can see the return change if `hx_trigger` is passed.


## Alpine.js

* Declare data. The data is available only for this node (container).
    ```html
    <div x-data="{count:0}">
    <div>
    ```
* Listening for envents. `x-on:click` means that when we click it will execute `count++`, a js expression. [Note: `@click` is equvalent to `x-on:click`]
    ```html
    <div x-data="{count:0}">
        <button x-on:click="count++">Increment</button>
    <div>
    ```
* `x-text` is an Alpine directive you can use to set the text content of an element to the result of a JavaScript expression.
    ```html
        <div x-data="{count:0}">
            <button x-on:click="count++">Increment</button>
            <span x-text="count"></span>
        <div>
    ```